In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pyFAI
import pyFAI.calibrant
import pyFAI.azimuthalIntegrator
import ipywidgets as widgets
from ipywidgets import interact
import time

In [2]:
def get_detector(det):
    return pyFAI.detector_factory(det)

def get_energy(wavelength):
    return 12.398*wavelength # returned in keV

def get_wavelength(energy):
    return 12.398/energy*1e-10 # returned in m

def get_radians(deg):
    return np.radians(deg)

In [3]:
def get_standard(std: str, energy: float):
    """
     supply standard as str and energy
     and get back pyFAI standard
    """
    standard = pyFAI.calibrant.get_calibrant(std)
    standard.set_wavelength(get_wavelength(energy))
    return standard

In [4]:
def geo_cone(X, Y, Z, rota, tilt, yoff, dist):
    # rotate the sample around y
    a = np.deg2rad(tilt) + np.deg2rad(rota)
    t = np.transpose(np.array([X,Y,Z]), (1,2,0))
    m = [[np.cos(a), 0, np.sin(a)],[0,1,0],[-np.sin(a), 0, np.cos(a)]]
    X,Y,Z = np.transpose(np.dot(t, m), (2,0,1))
    # compensate for tilt
    comp = np.deg2rad(tilt) * dist
    return Y,X+comp-yoff,Z

In [5]:
# initiate detector and geometry
#detector = get_detector('eiger2_4m')
#detector.getPyFAI()
#it is being done inside the plot currently

In [6]:
def plot_image(det, 
         method='pyFAI', 
         standard='Si', 
         energy=20.5, 
         dist=0.075, 
         offset1=0.08, 
         offset2=0.08, 
         rot=0, 
         tilt=0):
    """
     plots the standard with the given parameters

     runs once per interaction
    """
    
    # setup a timer
    t0 = time.perf_counter()
    
    # setup the detector
    detector = get_detector(det)
    print(f'Using detector {detector.get_name()}')
    print(f'Time to init detector: {time.perf_counter()-t0} s')
    
    # init the image with zeros
    image = np.zeros(detector.max_shape)
    
    # check if gaps exist and mask them if present
    # TODO: this currently fails for PE as .copy() is
    # not defined
    mask = detector.mask.copy()
    #print(f'Init mask: {time.perf_counter()-t0} s')
    if mask is not None:
        #image[np.where(mask)] = -1 # dummy value for non-existant pixels
        print(f'Mask out gaps: {time.perf_counter()-t0} s')
        
        
    fig, ax = plt.subplots(figsize=(12,12))
    ax.set(xticks=[], yticks=[], title=f'{detector.get_name()} with {standard}')
    if method == 'pyFAI':        
        std = get_standard(standard, energy)
    
        geom = pyFAI.azimuthalIntegrator.AzimuthalIntegrator(dist=dist, 
                                                         wavelength=get_wavelength(energy),
                                                         poni1=offset1,
                                                         poni2=offset2,
                                                         rot1=get_radians(rot),
                                                         rot2=get_radians(tilt),
                                                         detector=detector
                                                        )
    
        image = std.fake_calibration_image(geom)
        ax.imshow(image, origin='lower')
        ax.imshow(mask, alpha=0.3, origin='lower')
        print(f'Finished plotting after: {time.perf_counter()-t0} s') 
    elif method == 'cones':
        """
         this should be implemented with LKR's Plot_det_geo.py
         that calculates cones rather than pyFAI.fake_calibration_image
        """
        """
        #ax.imshow(mask, alpha=0.3, origin='lower')   
        # calculate physical sizes of detector in 
        # horizontal (x) and vertical (y) direction
        detx = detector.shape[0]*detector.pixel1
        dety = detector.shape[1]*detector.pixel2
        
        # figure out max dimension
        gridmax = max(detx, dety)
        
        # prepare grids within the detector
        steps = 50 # number of steps in the grid
        xmesh = np.linspace(-gridmax, gridmax, steps)
        ymesh = np.linspace(-gridmax, gridmax, steps)
        
        scale = 1
        X, Y = np.meshgrid(xmesh, ymesh)
        
        #2:_stl*4*np.pi
        Z = np.sqrt(X**2+Y**2)*5 #2.10 in Q
        
        X, Y, Z = geo_cone(X, Y, Z, 0, 0, 0, dist)
        ax.contour(X, Y, Z, [dist], 
                   #colors=plo.cont_orig_color, 
                   #alpha=plo.cont_orig_alpha
                  )
        #ax.imshow(mask, alpha=0.3, origin='lower')   
        """
        pass

    

   

    #return energy


In [7]:
interact(plot_image, 
         det=['Eiger2 CdTe 4M', 'Pilatus CdTe 2M', 'Perkin detector'], 
         method=['pyFAI', 'cones'],
         standard=['Si', 'Ni'],
         energy=(10, 35),
         dist=(0.05, 0.3, 0.001),
         offset1=(0, 0.5, 0.001),
         offset2=(0, 0.5, 0.001),
         rot=(0,45),
         tilt=(0,45)
         )

interactive(children=(Dropdown(description='det', options=('Eiger2 CdTe 4M', 'Pilatus CdTe 2M', 'Perkin detect…

<function __main__.plot_image(det, method='pyFAI', standard='Si', energy=20.5, dist=0.075, offset1=0.08, offset2=0.08, rot=0, tilt=0)>